In [5]:
using JuMP
using CPLEX
using CPUTime
using Graphs

In [1]:
struct Clients
    nb_clients
    X
    Y
    h
    L
    L0
    d
end

In [2]:
function Read_PRP_instance(filename)
    hashmap = Dict()
    open(filename) do f
        
        demands = false
        listing_active = false
        current_value = 0
        
        nbc = 0
        X = Array{Float64}(undef, 0)
        Y = Array{Float64}(undef, 0)
        h = Array{Float64}(undef, 0)
        L = Array{Float64}(undef, 0)
        L0 = Array{Float64}(undef, 0)
        d = Array{Array{Float64}}(undef, 0)
        
        for (i, line) in enumerate(eachline(f))
            x = split(line, " ")
            deleteat!(x, findall(e->e=="", x))
            current_item = x[1]
            
            if(current_item == "0" || listing_active) # Liste de clients / demandes
                listing_active = true
                if(current_item == "d")
                    demands = true
                elseif(demands) # Enregistrement des demandes
                    push!(d, parse.(Float64, x))
                else # enregistrement des clients
                    nbc += 1
                    push!(X, parse(Float64, x[2]))
                    push!(Y, parse(Float64, x[3]))
                    push!(h, parse(Float64, x[6]))
                    push!(L, parse(Float64, x[8]))
                    push!(L0, parse(Float64, x[10]))
                end
            else # variables générales du problème
                try
                    current_value = parse(Int64, x[2])
                catch err
                    current_value = 1^10
                end
                hashmap[current_item] = current_value
            end
        end
        C = Clients(nbc, X, Y, h, L, L0, d)
        hashmap["Clients"] = C
    end 
    return hashmap
end

Read_PRP_instance (generic function with 1 method)

In [3]:
h = Read_PRP_instance("./PRP_instances/A_014_#ABS1_15_1.prp")

Dict{Any, Any} with 9 entries:
  "f"       => 3000
  "Q"       => 322
  "C"       => 1
  "k"       => 2085
  "Type"    => 1
  "u"       => 30
  "l"       => 6
  "Clients" => Clients(15, [143.0, 89.0, 76.0, 285.0, 401.0, 16.0, 267.0, 249.0…
  "n"       => 14

In [4]:
function Resolve_PRP(G)
    LP = Model(CPLEX.Optimizer)
    
    # @variable(LP, x[1:G.nb_points, 1:G.nb_points], Bin)  # We will only use x_{ij} with i<j
    # objective function
    # @objective(LP, Min, sum((sum(x[i, j] * c[i, j]) for j = i+1:G.nb_points) for i = 1:G.nb_points ) )
    

    # Edge constraints
    # for i in 1:G.nb_points
    #    @constraint(LP, (sum(x[j, i] for j in 1:i-1)) + (sum(x[i, j] for j in i+1:G.nb_points))== 2)
    # end   

end

Resolve_PRP (generic function with 1 method)